# CityBike Wien Datensetanalye für die Jahre 2016 und 2017

== Inhalt und Struktur dieses Notebooks 



## Datenset importieren bereinigen und transformieren

Die Daten werden zuerst eingelesen und nötige Transformationen werden vorgenommen. 

In erste Linie werden alle Fahrten die unter zwei Minuten dauern und an der gleichen Station enden aus dem Datenset herausgenommen da man davon ausgehen kann, dass der Nutzer das System nicht benutzt hat. 

In [2]:
%matplotlib inline
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
#Warnungen seitens Jupyter sollen ignoriert werden
warnings.filterwarnings('ignore')

df2016 = pd.read_csv('CityBikeWien/2016trips.csv', error_bad_lines=False, parse_dates=['Startdatum', 'Enddatum'], delimiter=';')
df2017 = pd.read_csv('CityBikeWien/2017trips.csv', error_bad_lines=False, parse_dates=['Startdatum', 'Enddatum'], delimiter=';')

Die beiden DataFrames für die Jahre 2016 und 2017 werden zusammengeführt

In [3]:
df_total = df2016.append(df2017)

Bei dem neuen DataFrame wird der Index neu gesetzt

In [4]:
df_total = df_total.reset_index(drop=True)

Eine neue Spalte für die Fahrtdauer wird hinzugefügt (Enddatum-Startdatum)

In [5]:
df_total['Fahrtdauer'] =  df_total['Enddatum'] - df_total['Startdatum']

Uns interessieren nur Fahrten die länger als zwei Minuten dauern und gleichzeitig nicht an der gleichen Station beendet wurden. Die Bereinigung wird für alle dreit Datasets vorgenommen.

In [19]:
df_total_cleansed = df_total[ (df_total['Fahrtdauer'] > pd.Timedelta('120 sec')) 
                            & (df_total['Anfangsstation_ID'] != df_total['Endstation_ID'])]

df2016            = df2016[ (df2016['Fahrtdauer'] > pd.Timedelta('120 sec'))
                            & (df2016['Anfangsstation_ID'] != df2016['Endstation_ID'])]

df2017            = df2017[ (df2017['Fahrtdauer'] > pd.Timedelta('120 sec'))
                            & (df2017['Anfangsstation_ID'] != df2017['Endstation_ID'])]

Das bereinigte DataFrame enthält insgesamt 1.897.764 Einträge für die Jahre 2016 und 2017 mit folgenden Datentypen

In [20]:
df_total_cleansed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1897764 entries, 1 to 2080071
Data columns (total 7 columns):
Startdatum           datetime64[ns]
Enddatum             datetime64[ns]
Anfangsstation_ID    int64
Endstation_ID        int64
Fahrrad_ID           int64
Subscriber_ID        object
Fahrtdauer           timedelta64[ns]
dtypes: datetime64[ns](2), int64(3), object(1), timedelta64[ns](1)
memory usage: 115.8+ MB


Erste Zeile des DataFrames als Referenz

In [21]:
df_total_cleansed.head(1)

,Startdatum,Enddatum,Anfangsstation_ID,Endstation_ID,Fahrrad_ID,Subscriber_ID,Fahrtdauer
1,2016-01-01 00:13:04,2016-01-01 00:38:05,1026,1100,2539,0x04084286D0CC7FA3973C16F8306802A1,00:25:01


## Finde die ersten 10 Benutzer nach Benutzung für den gesamten Zeitraum (2016+2017)


In der folgenden Zelle wird das DataFrame anhand der 'Subscriber_ID' gruppiert. 

Darauffolgend wird eine Aggregatfunktion (np.size) auf das gruppierte Objekt angewendet um die Anzahl der Fahrten je Subscriber zu erhalten. Somit erhält man ein Series-Objekt welches sortiert werden muss. 

Um die Handhabung zu erleichtern wird das Series-Objekt zu einem DataFrame mit einer einzigen Spalte transformiert.

In [22]:
grouped_by_subscriber_groups              = df_total.groupby('Subscriber_ID')
aggregated_by_subscriber_series           = grouped_by_subscriber_groups['Fahrrad_ID'].agg(np.size)
aggregated_sorted_by_subscriber_series    = aggregated_by_subscriber_series.sort_values(ascending=False)
aggregated_by_subscriber_dataframe        = aggregated_sorted_by_subscriber_series.to_frame('ANZAHL_FAHRTEN')

Die ersten zehn Subscriber werden extrahiert und zudem wird eine Spalte mit dem Rang hinzugefügt. Danach wird das DataFrame ausgegeben. 

In [23]:
aggregated_by_subscriber_dataframe_top_10 = aggregated_by_subscriber_dataframe[:10]
aggregated_by_subscriber_dataframe_top_10['RANG'] = np.arange(1,11) 
aggregated_by_subscriber_dataframe_top_10

,ANZAHL_FAHRTEN,RANG
Subscriber_ID,,
0x30EF7C4D0C1211B47ADBA3295EF47C61,1790,1
0xB9AF912E74C4F6602CF849721E28B621,1400,2
0xAC44D643FD2F8AE0CCD64B35A269DCC1,1297,3
0x30F9A85D6ADB8CFA380C4EA7D77C574E,1281,4
0xCA316F5E99321FD79D33C3CAB1000DA5,1193,5
0xE88FAEE35C9817B6BD91961D8DD186E0,1028,6
0x62B4268B5D5AB0FD8B0FDC93D79498DD,1025,7
0x24384492DBD8ACA63F4C29FC8BC3B36A,1011,8
0xD372854C5186AB811AEE55325547CC85,1009,9


## Was hat der Benutzer mit den meisten Fahrten für die Nutzung des Systems bezahlt?

Der 'Citybike Wien'-Nutzer welcher das System innerhalb der beiden Jahren am meisten genutzt hat, fuhr insgesamt 1790 Fahrten. Es wäre interessant zu sehen was ihn die Nutzung des Systems über die Zeitdauer der beiden Jahre gekostet hat. 

Hierfür ist es nötig den Kostenplan von Citybike Wien zu kennen. Dieser ist progressiv eingestellt, das heißt, dass die erste Stunde kostenlos ist. Die zweite Stunde kostet einen Euro, die dritte das doppelte und die vierte Stunde dann vier Euro. Jede weitere Stunde kostet jeweils vier Euro. 

Wir suchen alle Fahrten für den Power-User heraus

In [24]:
df_user_1 = df_total[df_total['Subscriber_ID'] == '0x30EF7C4D0C1211B47ADBA3295EF47C61']

Wie oft hat der Nutzer das Limit der einen Stunde überschritten? Insgesamt 98 mal.

In [25]:
df_user_1[df_user_1['Fahrtdauer'] > pd.Timedelta('60 min')].size

98

Gab es Fälle bei denen die Fahrt länger als zwei Stunden dauerte? Nein

In [26]:
df_user_1[df_user_1['Fahrtdauer'] > pd.Timedelta('120 min')].size

0

Insgesamt fielen 98 kostenpflichtige Fahrten an welche weniger als zwei Stunden gedauert haben. Somit hat der Anwender insgesamt 98 Euro für die Nutzung des Systems für die Zeitdauer der beiden Jahren bezahlt da jede Fahrt genau einen Euro gekostet hat.

Im Schnitt hat der oder die NutzerIn ~ 5,5 Cent für jede Fahrt bezahl.


## Welchen Umsatz hat das System für das Jahr 2016 bzw. 2017 generiert?

Es wäre interessant zu sehen wie viel Umsatz alleine durch die Nutzung des Systems entstanden ist. 

Der Betreiber von CityBike Wien, die Gewista GmbH wählt ein hybrides Finanzierungsmodell. Das heißt, dass sich das System nicht nur aus den Beiträgen der einzelnen Nutzer finanziert, sondern auch aus Werbungen welche auf den Fahrrädern angezeigt werden. Inwieweit oder ob das System durch die Stadt Wien subventioniert wird ist nicht bekannt. 

Wir untersuchen nun die Einnahmen welche durch die Nutzergemeinde des Systems entsteht. Dies ist durch die Kenntniss des Kostenmodells und der Länge der einzelnen Fahrten möglich. Die Anmeldegebühr von einem Euro wird nicht mit in die Berechnung einbezogen da aus dem Datenset nicht bekannt ist wann sich der Nutzer das erste Mal registriert hat. Somit könnte es sein, dass die Anmeldegebühr nicht in den Jahren 2016 oder 2017 verrechnet wurde. 

In [27]:
df2016['Fahrtdauer'] = df2016['Enddatum'] - df2016['Startdatum']
df2017['Fahrtdauer'] = df2017['Enddatum'] - df2017['Startdatum']

In [79]:
def calculate_fare(fahrtdauer):
    ssddsg=  math.ceil(fahrtdauer.total_seconds() // 3600)
    if(fahrtdauer < pd.Timedelta('60 min')):        return int(0)
    elif( (fahrtdauer > pd.Timedelta('60 min')) & (fahrtdauer < pd.Timedelta('120 min'))):
        return int(1)
    elif( (fahrtdauer > pd.Timedelta('120 min')) & (fahrtdauer < pd.Timedelta('180 min'))):
        return int(3)
    elif( (fahrtdauer > pd.Timedelta('180 min')) & (fahrtdauer < pd.Timedelta('240 min'))):
        return int(7)
    elif((fahrtdauer > pd.Timedelta('240 min'))):
        return (int(7) +  ((math.ceil(fahrtdauer.total_seconds() // 3600))-3)*4)
    #a case where the bike is not returned within six days 

In [80]:
df2016_test = df2016

In [81]:
df2016_test['Fahrtpreis'] = df2016_test['Fahrtdauer'].apply(calculate_fare)

In [82]:
df2016_test[ df2016_test['Fahrtdauer'] > pd.Timedelta('21000 sec') ]

,Startdatum,Enddatum,Anfangsstation_ID,Endstation_ID,Fahrrad_ID,Subscriber_ID,Fahrtdauer,Fahrtpreis
71,2016-01-01 05:03:33,2016-01-01 12:05:51.000,1046,1085,3302,0xE67FE030A7B68D78E684228397BE7688,0 days 07:02:18,23.0
77,2016-01-01 05:58:09,2016-01-01 13:50:14.000,1024,1105,2869,0x68574B0A9190005828ABE9C9425E3020,0 days 07:52:05,23.0
5621,2016-01-11 22:08:35,2016-01-12 04:14:24.000,1063,1055,3598,0x5DF8DB344020E4519A96DB2A8B74A8D5,0 days 06:05:49,19.0
10794,2016-01-15 22:12:55,2016-01-16 10:58:06.000,1076,1051,2909,0xBE269AA7E7B39B363452D52523C11E39,0 days 12:45:11,43.0
11548,2016-01-16 18:59:58,2016-01-17 00:59:53.000,1086,1137,2587,0x97A8F7728EC0117F42A33A5E74F51864,0 days 05:59:55,15.0
11798,2016-01-17 02:07:38,2016-01-17 12:59:27.000,1116,1045,3685,0xF589445422E364819CABCC22CD3164D4,0 days 10:51:49,35.0
11919,2016-01-17 10:12:40,2016-01-17 16:48:10.000,1125,1109,2900,0x4A34CCAD279BDF5E069544A3CD4D8005,0 days 06:35:30,19.0
13618,2016-01-19 09:46:34,2016-01-19 23:47:32.000,1044,1067,3252,0xDC9B5193E7C0266591D1397129E4ABAC,0 days 14:00:58,51.0
19428,2016-01-26 15:04:02,2016-01-27 12:39:51.000,1146,1038,3529,0x6501230B41DE368B04E05ED6F31DE201,0 days 21:35:49,79.0
22488,2016-01-28 12:19:22,2016-01-28 18:39:49.000,1134,1034,3621,0x7D4E4B170731B667DDD5165852C9769B,0 days 06:20:27,19.0
